In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
from nanomesh.volume import Volume

In [ ]:
vol = Volume.load('sample_data.npy')

In [ ]:
vol.show_slice()

In [ ]:
vol.show_volume(renderer='itk')

In [ ]:
vol.show_volume(renderer='ipyvolume')

In [ ]:
from nanomesh import sitk_filter

vol = vol.apply(sitk_filter.gaussian_filter, sigma=5.0)
vol.show_slice(title='Gaussian filter')

In [ ]:
inner = vol.apply(sitk_filter.binary_threshold, 
                  lowerThreshold=0, 
                  upperThreshold=125, 
                  insideValue=1, 
                  outsideValue=0,
                  )

inner.show_slice(title='Inner regions')

In [ ]:
inner_mesh = inner.generate_mesh(h=(1,1,1))
inner_mesh

In [ ]:
import pyvista as pv
pv.plot_itk(inner_mesh)

In [ ]:
import meshio
meshio.write('inner.vtk',inner_mesh)

In [ ]:
inner_mesh = pv.read('inner.vtk')

bounds = [100,200,100,200,100,200]
clipped = inner_mesh.clip_box(bounds)

pv.set_plot_theme("document")

p = pv.Plotter(notebook=True)

p.add_mesh(inner_mesh, style='wireframe', color='black')
p.add_mesh(clipped,  color='lightgrey')
p.show()

In [ ]:
two_phase = vol.apply(sitk_filter.binary_threshold,
                      lowerThreshold=0, 
                      upperThreshold=125, 
                      insideValue=1, 
                      outsideValue=2)

vol.show_slice(overlay=two_phase.image, title="Binary Thresholding")

In [ ]:
two_phase.show_volume(renderer='ipyvolume')

In [ ]:
two_phase_mesh = two_phase.generate_mesh()

In [ ]:
pv.plot_itk(two_phase_mesh)

In [ ]:
import meshio
meshio.write('twophase.vtk', two_phase_mesh)

In [ ]:
mesh = pv.read('twophase.vtk')

bounds = [100,200,100,200,100,200]
clipped = mesh.clip_box(bounds)
pv.set_plot_theme("document")

p = pv.Plotter(notebook=True)
p.add_mesh(mesh, style='wireframe', color='black')
p.add_mesh(clipped)
p.show(use_ipyvtk=False)